In [ ]:
%md
### Kardiaflow - Gold: Provider Rolling Spend

**Source:** `kardia_silver.silver_claims_enriched`

**Targets:**
`gold_provider_daily_spend`
`gold_provider_7d_spend`

**Trigger:** Full snapshot overwrite each run.

**Purpose:** Summarize provider claim spend over time. Produces daily totals per provider and rolling 7-day spend metrics (total and average) for short-term financial trend monitoring.


In [ ]:
from kflow.config import GOLD_DB, GOLD_DIR
from kflow.notebook_utils import init

init()

In [0]:
# Ensure Gold DB exists
spark.sql(f"CREATE DATABASE IF NOT EXISTS {GOLD_DB}")
spark.sql(f"USE {GOLD_DB}")

In [0]:
%sql  
-- Daily provider spend  
CREATE OR REPLACE TABLE gold_provider_daily_spend
USING DELTA
LOCATION '${GOLD_DIR}/gold_provider_daily_spend'
AS
SELECT
  provider_id,
  CAST(claim_date AS DATE) AS spend_date,
  SUM(claim_amount)        AS daily_spend
FROM kardia_silver.silver_claims_enriched
GROUP BY provider_id, CAST(claim_date AS DATE);

In [0]:
%sql
-- 7-day rolling spend & average per provider
CREATE OR REPLACE TABLE gold_provider_7d_spend
USING DELTA
LOCATION '${GOLD_DIR}/gold_provider_7d_spend'
AS
WITH daily_spend_per_provider AS (
  -- sum spend per provider per day
  SELECT
    provider_id,
    CAST(claim_date AS DATE) AS spend_date,
    SUM(claim_amount) AS daily_spend
  FROM kardia_silver.silver_claims_enriched
  GROUP BY provider_id, CAST(claim_date AS DATE)
),

rolling_metrics AS (
  -- 7-day calendar rolling totals and average
  SELECT
    provider_id,
    spend_date,
    daily_spend,
    SUM(daily_spend) OVER (
      PARTITION BY provider_id
      ORDER BY CAST(spend_date AS TIMESTAMP)
      RANGE BETWEEN INTERVAL 6 DAYS PRECEDING AND CURRENT ROW
    ) AS rolling_7d_spend,
    ROUND(
      AVG(daily_spend) OVER (
        PARTITION BY provider_id
        ORDER BY CAST(spend_date AS TIMESTAMP)
        RANGE BETWEEN INTERVAL 6 DAYS PRECEDING AND CURRENT ROW
      ),
      2
    ) AS rolling_7d_avg
  FROM daily_spend_per_provider
)

SELECT *
FROM rolling_metrics;

In [0]:
%sql  
-- Preview: daily spend  
SELECT *  
FROM gold_provider_daily_spend
ORDER BY spend_date DESC  
LIMIT 10;  

In [0]:
%sql  
-- Preview: 7‑day rolling spend  
SELECT *  
FROM gold_provider_7d_spend
ORDER BY spend_date DESC  
LIMIT 10;